# Question 1: Extraction des taux zéro-coupon

Date actuelle: 18 septembre 2025

Les coupons obligataires sont semi-annuels.

In [1]:
import numpy as np
import pandas as pd

In [2]:
# Rendements observés sur le marché (en %)
rendements = {
    '4 sem': 4.04,
    '3 mois': 3.89,
    '6 mois': 3.72,
    '1 an': 3.47,
    '2 ans': 3.57,
    '3 ans': 3.55,
    '5 ans': 3.67,
    '7 ans': 3.86,
    '10 ans': 4.11,
    '20 ans': 4.68
}

# Conversion en années pour chaque maturité
maturites = {
    '4 sem': 4/52,
    '3 mois': 0.25,
    '6 mois': 0.5,
    '1 an': 1.0,
    '2 ans': 2.0,
    '3 ans': 3.0,
    '5 ans': 5.0,
    '7 ans': 7.0,
    '10 ans': 10.0,
    '20 ans': 20.0
}

print("Données du marché:")
for periode, taux in rendements.items():
    print(f"{periode}: {taux}%")

Données du marché:
4 sem: 4.04%
3 mois: 3.89%
6 mois: 3.72%
1 an: 3.47%
2 ans: 3.57%
3 ans: 3.55%
5 ans: 3.67%
7 ans: 3.86%
10 ans: 4.11%
20 ans: 4.68%


In [ ]:
# Méthode de bootstrapping pour extraire les taux zéro-coupon
# Les bons du Trésor (≤ 1 an) sont déjà des zéro-coupons
# Les obligations (≥ 2 ans) ont des coupons semi-annuels

taux_zero = {}

# Pour les bons du Trésor (≤ 1 an), le rendement = taux zéro-coupon
taux_zero['4 sem'] = rendements['4 sem']
taux_zero['3 mois'] = rendements['3 mois']
taux_zero['6 mois'] = rendements['6 mois']
taux_zero['1 an'] = rendements['1 an']  

print("\nTaux zéro-coupon extraits:")
print(f"4 semaines: {taux_zero['4 sem']:.4f}%")
print(f"3 mois: {taux_zero['3 mois']:.4f}%")
print(f"6 mois: {taux_zero['6 mois']:.4f}%")
print(f"1 an: {taux_zero['1 an']:.4f}%")


Taux zéro-coupon extraits:
4 semaines: 4.0400%
3 mois: 3.8900%
6 mois: 3.7200%
1 an: 3.4700%


In [ ]:
# Bootstrapping pour les maturités plus longues (2, 3, 5, 7, 10, 20 ans)
periodes_longues = ['2 ans', '3 ans', '5 ans', '7 ans', '10 ans', '20 ans']

# Stocker tous les taux zéro en format annualisé (composé semestriellement)
z_dict = {
    4/52: taux_zero['4 sem'] / 100,
    0.25: taux_zero['3 mois'] / 100,
    0.5: taux_zero['6 mois'] / 100,
    1.0: taux_zero['1 an'] / 100
}

for periode in periodes_longues:
    T = maturites[periode] 
    y = rendements[periode] / 100
    coupon = (y / 2) * 100
    
    # Calculer la valeur actuelle de tous les coupons sauf le dernier
    n_coupons = int(T * 2) 
    va_coupons = 0
    
    for i in range(1, n_coupons):
        t = i * 0.5
        # Trouver le taux zéro approprié (interpolation si nécessaire)
        if t in z_dict:
            z_t = z_dict[t]
        else:
            # Interpolation linéaire simple
            temps_disponibles = sorted([k for k in z_dict.keys() if k <= t])
            if temps_disponibles:
                t_avant = max(temps_disponibles)
                temps_apres = [k for k in z_dict.keys() if k > t]
                if temps_apres:
                    t_apres = min(temps_apres)
                    # Interpolation linéaire
                    poids = (t - t_avant) / (t_apres - t_avant)
                    z_t = z_dict[t_avant] * (1 - poids) + z_dict[t_apres] * poids
                else:
                    z_t = z_dict[t_avant]
            else:
                z_t = 0.04  # Valeur par défaut
        
        va_coupons += coupon / (1 + z_t/2)**(i)
    
    # Résoudre pour le taux zéro à maturité T
    # 100 = va_coupons + (coupon + 100) / (1 + z_T/2)^(2T)
    valeur_finale = coupon + 100
    va_restante = 100 - va_coupons
    
    z_T_semi = (valeur_finale / va_restante)**(1/(n_coupons)) - 1
    z_T = z_T_semi * 2
    
    taux_zero[periode] = z_T * 100
    z_dict[T] = z_T

print("\nTous les taux zéro-coupon extraits:")
for periode in rendements.keys():
    print(f"{periode}: {taux_zero[periode]:.4f}%")


Tous les taux zéro-coupon extraits:
4 sem: 4.0400%
3 mois: 3.8900%
6 mois: 3.7200%
1 an: 3.4700%
2 ans: 3.5716%
3 ans: 3.5497%
5 ans: 3.6803%
7 ans: 3.8912%
10 ans: 4.1847%
20 ans: 5.0901%


In [8]:
# Tableau récapitulatif
df = pd.DataFrame({
    'Maturité': list(rendements.keys()),
    'Rendement du marché (%)': list(rendements.values()),
    'Taux zéro-coupon (%)': [taux_zero[p] for p in rendements.keys()]
})

print("\nTableau récapitulatif:")
print(df.to_string(index=False))


Tableau récapitulatif:
Maturité  Rendement du marché (%)  Taux zéro-coupon (%)
   4 sem                     4.04              4.040000
  3 mois                     3.89              3.890000
  6 mois                     3.72              3.720000
    1 an                     3.47              3.470000
   2 ans                     3.57              3.571556
   3 ans                     3.55              3.549651
   5 ans                     3.67              3.680329
   7 ans                     3.86              3.891213
  10 ans                     4.11              4.184747
  20 ans                     4.68              5.090134
